# Main Model Construction

In [ ]:
import numpy as np
import random

##import 12-TET pitch set data -- already curated as a set
FULL_PITCH_SETS = 'full_pitch_sets.txt'

##format pitch sets: create an array filled with each pitch set
def create_pitch_sets(sets_filename):
    with open(sets_filename) as file:
        lines = [set(line.rstrip()) for line in file]
    return lines

##build the backbone of the markov model -- distance matrix with probabilities based on 
##intersection of pitch classes; normalize to sum to 1; cumulative sum to prepare for random
##walk
def create_distance_matrix(pitch_sets):
    distance_matrix = []
    for i in range(len(pitch_sets)):
        row = []
        for j in range(len(pitch_sets)):
            row.append(len(pitch_sets[i].intersection(pitch_sets[j])))
        row_sum = sum(row)
        row = np.array(row,dtype='f')/row_sum
        row = np.cumsum(row)
        distance_matrix.append(list(row))
    return(distance_matrix)

In [ ]:
##main drunkards walking music code

##Inputs: list of pitch sets; distance matrix (of probabilities)
##Function: randomly chooses a starting point; searches starting node's distance row and
##          chooses next node based on probabilities
##Output: an array documenting the random walk across the markov space

##Note: ITERATIONS is a global variable; could be converted into an input
ITERATIONS = 8

def create_drunkards_walking_music(pitch_sets, distance_matrix):
    current_set = random.randint(0,len(pitch_sets)-1)
    drunkards_walking_music = [pitch_sets[current_set]]
    for i in range(ITERATIONS):
        r = random.uniform(0,1)
        j = 0
        while distance_matrix[current_set][j] < r:
            j +=1
        drunkards_walking_music.append(list(pitch_sets[j]))
        current_set = j
    return(drunkards_walking_music)

In [ ]:
##Composition helper functions

##Array of frequency values (will be helpful later)
FREQUENCY_VALUES = [
    261.626,
    277.183,
    293.665,
    311.127,
    329.628,
    349.228,
    369.994,
    391.995,
    415.305,
    440,
    466.164,
    493.883
]

##Self defined flatten function for ease of use
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
##CREATE SOUND

##INPUT: a pitch set, its octave displacement (fraction or int), a duration (as an integer),
##       sampling frequency and a number of harmonics
##FUNCTION: - randomly choose one element of the pitch set
##          - convert pitch set from string to an array of ints
##          - create time vector length of'duration' seconds with 'fs' samples per second
##          - create fundamental oscillator by converting pitch class into frequency value
##          - add overtone oscillators following same process but increase octave and decrease
##            amplitude
##          - sum these oscillators together to create a complex tone
##OUTPUT: an np.array representing a sum of oscillators within the pitch set

##NOTE: okay I think the 'lift' component of the overtones is wrong; it's adding octave way too
##      fast, which I think will cause aliasing. I don't think aliased tones necessarily
##      continue to exist in the pitch set -- THIS HAS TO BE FIXED
##
##      Second note: zip(*) takes each element of the oscillators and zips them into tuples
##                   this allows the element-wise summation to happen at the return of the function
def create_sound(pitch_set, octave, duration, fs, harmonics):
    pitch = random.sample(pitch_set, k=1)[0]
        
    if pitch == 'A':
        pitch = 10
    elif pitch == 'B':
        pitch = 11
    else:
        pitch = int(pitch)
            
    time = np.arange(0, duration, 1/fs)
    
    #fundamental
    fundamental = np.cos(2*np.pi*octave*FREQUENCY_VALUES[pitch]*time)
    
    #harmonics
    lift = 1
    amp = 1
    overtones = []
    for i in range(harmonics):
        over_pitch = random.sample(pitch_set, k=1)[0]
        
        if over_pitch == 'A':
            over_pitch = 10
        elif pitch == 'B':
            over_pitch = 11
        else:
            over_pitch = int(pitch)
        
        amp = amp*.75
        lift = lift*(i+1)
        overtones.append(amp*np.cos(2*np.pi*lift*octave*FREQUENCY_VALUES[over_pitch]*time))
        
    overtones.append(fundamental)
    
    to_sum = zip(*overtones)
    
    return [sum(item) for item in to_sum]

In [ ]:
##CREATE COMPLEX LAYER

##INPUT: a pitch set (string); a duration (int); a local duration, smaller than duration (int)
##       an amplitude value (float); an octave (float); sampling frequency; complexity (int)
##FUNCTION: - divides duration by local duration to determine number of iterations
##          - for each iteration it creates a time vector, creates a window (envelope) and
##            appends to the output array the output of create_sound (a complex tone)
##          - flattens the list of complex sounds into a long list of multiple complex sounds
##OUTPUT: a list of amplitude values representing a complex layer of a landscape
def create_complex_layer(pitch_set, duration, local_dur, amplitude, octave, fs, complexity):
    tones = []
    for i in range(round(duration/local_dur)):
        
        time = np.arange(0,local_dur,1/fs)
        window = np.hamming(len(time))
        #window = random_adsr(len(time), .05, .25, .65, .05, fs)
        tones.append(window*amplitude*create_sound(pitch_set, octave, local_dur, fs, complexity))
        
    return flatten(tones)

In [ ]:
##VARIABLY SIZED LANDSCAPE

##INPUT: a pitch set (string); local_dur (int); amplitude (float); octave (float); sampling frequency
##       number of layers (int); possible complexity range (array of ints)
##FUNCTION: - creates a list of durations with each successive duration being half of the previous
##          - creates a list of amplitudes with each successive one being deteremined by 1/n
##          - chop of these lists to contain local_dur and all durations/amplitudes smaller than it
##          - for each layer in the number of layers, set overall duration, set local duration
##            set octave, set amplitude and randomly choose complexity from range; then run
##            create_complex_layer with these values and append to output list
##          - zip the list of layers into one big landscape
##OUTPUT: the complete landscape made up of many layers of oscillators
##
##NOTE: This is pretty poorly designed. The durations/amplitudes are hard coded and builds into
##      each successive layer that the amplitude will be 1/n and duration will be half of the previous
##      would be better to make this a variable at the main function level
def variably_sized_landscape(pitch_set, local_dur, amplitude, octave, fs, layers, complexities):
    landscape = []
    
    DURS = [local_dur/(2**x) for x in range(0,11)]
    AMPS = [amplitude/x for x in range(1,12)]
    
    for i in range(len(DURS)):
        dur = DURS[i]
        if dur == local_dur:
            durations = DURS[i:]
            amplitudes = AMPS[i:]
    
    if layers > len(durations):
        print('Please adjust your layers to be smaller than the number of durations')
    
    for i in range(layers):
        duration = durations[0]
        local_dur = durations[i]
        octav = octave*(2**i)
        amplitude = amplitudes[i]
        complexity = random.sample(complexities, k=1)[0]
        landscape.append(create_complex_layer(pitch_set, duration, local_dur, amplitude, octav, fs, complexity))
    
    to_sum = zip(*landscape)
    return [sum(item) for item in to_sum]

In [ ]:
#%%timeit -r 1 -n 1

## Composition
from IPython.display import Audio

SAMPLING_FREQUENCY = 44100

if __name__ == "__main__":
    
## preporatory work: load pitch sets; build distance matrix; create random walk
## NOTE: sorted drunkards is simply sorting the pitch set to make it so it's in order of
##       ascending pitch class
    pitch_sets = create_pitch_sets(
        FULL_PITCH_SETS
    )
    distance_matrix = create_distance_matrix(pitch_sets)
    drunkards_walking_music = create_drunkards_walking_music(
        pitch_sets, 
        distance_matrix
    )
    sorted_drunkards = [sorted(item) for item in drunkards_walking_music]
    
## create arrays for multichannel output; in this case, stereo
    oscillator_list_l = []
    oscillator_list_r = []
    
##BULK OF COMPOSITIONS
##Rhythm: for each channel, hard code an array of lengths
    counts = [1,2,3,4,5,6,7,8,9,10,11,11,10,9,8,7,6,5,4,3,2,1]
    counts = [1,2,3,4,5,4,3,2,1]
    #counts2 = [8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8]
    count = 1

##Octave displacement: create a set of octaves; 1 is A440, so this goes both above and below
    octaves = [
        .0625,
        .125,
        .25,
        .5,
        1,
        2,
        4,
        8,
        16,
        32,
        64
        ]
    
##Rhythm
    time1 = [x + random.randint(1,20) for x in np.zeros(ITERATIONS+1)]
    time2 = random.sample(time1, k=len(time1))
    
    for pitch_set in sorted_drunkards:
        octave1 = random.sample(octaves, k=1)[0]
        #octave2 = random.sample(octaves, k=1)[0]
        oscillator_list_l.append(variably_sized_landscape(pitch_set, 20, 1/counts[count-1], .0625, 44100, counts[count-1], [(item+1) for item in list(range(counts[count-1]))]))
        #oscillator_list_r.append(variably_sized_landscape(pitch_set, time2[count-1], 1/octave2*8, octave2, 44100, counts2[count-1], [(item+1) for item in list(range(counts2[count-1]))]))
        count += 1
    
    comp_length = len(flatten(oscillator_list_l))
    #WINDOW1 = random_adsr(comp_length, .2, .2, .3, .3, 44100)
    WINDOW2 = np.hamming(len(flatten(oscillator_list_l)))
    
Audio([WINDOW2*flatten(oscillator_list_l)],rate=SAMPLING_FREQUENCY)

# UPDATED DISTANCE MATRIX

In [ ]:
#UPDATED DISTANCE METRIC INCORPORATING INTERSECTION, MANHATTAN & EUCLIDEAN
#UPDATED ALSO TO INCORPORATE VARIABLE INPUT (INTEGER, RATIO, HERTZ)

def create_new_distance_matrix(pitch_sets):
    distance_matrix = []
    for i in range(len(pitch_sets)):
        row = []
        static_set = pitch_set[i]
        for j in range(len(pitch_sets)):
            ##comparison algorithm changed
            comp_set = pitch_sets[j]
            static_leftover, comp_leftover = remainder_after_intersection(static_set, comp_set)
            ##if the remainder is empty for both (i.e. they are the same set), give a value of 1
            if not static_leftover and not comp_leftover:
                ## score = 0 means it could never go to itself -- might be worth considering
                ## score = 100 means very low probability
                ## score = 1 means very high probability
                score = 1
            else:
                ## if the two leftovers are of different sizes -- what should the score be?
                ## Case1: extra notes are treated as nothing (generating voices does not take energy)
                ## Case2: extra notes need to find the nearest neighbor to collapse or expand to (all voices are maintained)
                ## Case3: unbalanced; if static > comp, note off events (losing a voice) don't take energy;
                ##        if comp > static, note on events (adding a voice) do take energy and need to have a neighbor to come from
                ## Case3 can be flipped if desired
                ##
                ## I think these rules make sense: if you can maintain common tones, do so. If you have leftover notes, existing voices
                ## should be kept existing. This means that even if there is a note in set1 that is closer to a remainder note in set2
                ## but it is not in the remainder of set1, then the leftover voices of both sets should be used to calculate distance.
                ## The only cases extenuating for this rule is if there is an addition or subtraction of an additional voice. In this case,
                ## any note in the opposite set can be used to "split" or "collapse" the voice.
                ## 
                ## It is important to note that this system cannot be extended to different voices of chords at the moment. For
                ## now, it assumes all chords are created equal in terms of voicing and instead cares about the collection of notes
                ## more than how they are sounded. This is definitely something left to future work, as it would make more sense to
                ## accomodate different voicings of chords (the distance function could be calculate simply by measuring the distance)
                ## in frequency as opposed to divisions of the octave.
                ##
                ## Another important note is that the assumptions of this model assume equal distance within different divisions of 
                ## different spaces. This is obviously not the case but is used as a jumping off point for the building of this system.
                ##
                ## all that being said, here is the implementation of the most rudimentary version of this algorithm
                if len(static_leftover) == len(comp_leftover):
                    ## nuance of being equal in length: if two notes are equidistant from one note, or if they are both close
                    ## to one note and not the other, how do we ensure the lowest possible score
                    for item in static_leftover:
                        return 0
                
            row.append(len(pitch_sets[i].intersection(pitch_sets[j])))
        row_sum = sum(row)
        row = np.array(row,dtype='f')/row_sum
        row = np.cumsum(row)
        distance_matrix.append(list(row))
    return(distance_matrix)

In [ ]:
import numpy as np

def find_map_and_score(set1, set2, TYPE, division = 12):
    #make sure smallest set is used for comparisons
    reverse_indicator = 1
    if len(set1) > len(set2):
        holder = set1[:]
        set1 = set2[:]
        set2 = holder[:]
        reverse_indicator = -1
        
    ##Initiliaze outputs
    mappings = []
    score = 0
    
    #don't overwrite set2
    comparison_set = set2[:]
    
    for note in set1:
        smallest_distance = np.inf
        for dest in comparison_set:
            #distance function needs to be defined for specific input and units
            dist = distance(note, dest, TYPE, division = division)
            if dist < smallest_distance:
                smallest_distance = dist
                to_note = dest
        tup_out = (note, to_note)
        score += smallest_distance
        comparison_set.remove(tup_out[1])
        mappings.append(tup_out[::reverse_indicator])
        
    leftover = comparison_set
    return (mappings, score, leftover)  

In [ ]:
set1 = [0, 1, 2]
set2 = [3, 4, 5]

def distance(note1, note2, note_type, division = 12):
    if note_type == "int":
        return min(abs(note1-note2), division - abs(note1-note2))
    elif note_type == "ratio":
        return abs(note1-note2)
    elif note_type == 'hertz':
        return abs(np.log2(note1) - np.log2(note2))

In [ ]:
## this function assumes equal friction for adding and removing voices
import itertools
import numpy as np

def minimum_distance_mapping(set1, set2, TYPE, division = 12):
    ## set 1 should be smaller than set2 if different sizes
    reverse_indicator = 1
    if len(set1) > len(set2):
        holder = set1[:]
        set1 = set2[:]
        set2 = holder[:]
        reverse_indicator = -1
    maps_and_scores = []
    set1_orderings = list(itertools.permutations(set1))
    for combination in set1_orderings:
        combination = list(combination)
        maps_and_scores.append(find_map_and_score(combination, set2, TYPE, division=division))
    
    lowest_score = np.inf
    for map_and_score in maps_and_scores:
        current_score = map_and_score[1]
        current_map = map_and_score[0]
        current_leftover = map_and_score[2]
        if current_score < lowest_score:
            lowest_score = current_score
            best_map = current_map
            leftover = current_leftover
    
    ## this is where it gets a little bit tricky again
    ## should I have the leftover notes be allowed to go to the same note in the other set?
    ## or should we only allow one additional voice be connected to a shared note
    ## the latter seems more reasonable, but theoretically both are possible
    
    ## I will do the latter for now but the difference in code is just changing one call to 
    ## find_map_and_score to multiple individual calls
    ## more efficient to do the latter actually, but could be a cooler compositional effect doing the former
    ## actually it's more efficient to do the former, as the latter requires more permutations to be done
    ## but that is okay for now
    
    if len(leftover) != 0:
        while len(best_map) != len(set2):
            leftover_orderings = list(itertools.permutations(leftover))
            leftover_maps_and_scores = []
            for combination in leftover_orderings:
                combination = list(combination)
                leftover_maps_and_scores.append(find_map_and_score(combination, set1, TYPE, division = division))

            leftover_lowest = np.inf
            for leftover_map_and_score in leftover_maps_and_scores:
                leftover_map = leftover_map_and_score[0]
                leftover_score = leftover_map_and_score[1]
                leftover_leftover = leftover_map_and_score[2]
                if leftover_score < leftover_lowest:
                    leftover_lowest_score = leftover_score
                    best_leftover_map = [tup[::-1] for tup in leftover_map]
                    leftover = leftover_leftover
            best_map = best_map + best_leftover_map
            lowest_score += leftover_lowest_score
    
    final_map = [tup[::reverse_indicator] for tup in best_map]
    final_score = lowest_score 
    
    return (final_map, final_score)

# EXPERIMENTS IN WINDOWING

In [ ]:
## experiments in windowing
## we have attack, decay, sustain and release
## each of these things should be an array of values that follow some equation
## attack -- should start low (0) and end high
## decay -- needs to start where attack ends and fall 
## sustain -- should start where decay ends and be steady state
## release -- should start where sustain ends and end low (0)
##
## use linspace/logspace/geomspace

import matplotlib.pyplot as plt

attack = np.geomspace(0.001, 1.0, num = 100, endpoint = False)
decay = np.geomspace(1.0, 0.65, num = 50, endpoint = False)

fs = 44100
time = np.arange(0, 1/fs*200, 1/fs)
sustain = .05*np.sin(2*np.pi*20000*time) + decay[-1]

release = np.geomspace(sustain[-1], 0.001, num = 75, endpoint = True)
plt.plot(np.concatenate((attack, decay, sustain, release)))

In [ ]:
## take length of array and divide it into four unequal parts
##
## get length of array -- should be an int
length = 100

attack_max_endpoint = length*.2
decay_max_endpoint = length*.1
sustain_max_endpoint = length*.6
release_max_endpoint = length*.1

attack_end = int(np.ceil(random.uniform(.5,1)*attack_max_endpoint))
decay_end = int(np.ceil(random.uniform(.5,1)*decay_max_endpoint))
sustain_end = int(np.ceil(random.uniform(.5,1)*sustain_max_endpoint))
release_end = 100 - (attack_end + decay_end + sustain_end)

print(attack_end)
print(decay_end)
print(sustain_end)
print(release_end)

attack = np.geomspace(0.001, random.uniform(.8,1), num = attack_end, endpoint = True)
decay = np.geomspace(attack[-1], random.uniform(.4,.8), num = decay_end, endpoint = False)

fs = 44100
time = np.arange(0, 1/fs*sustain_end, 1/fs)
sustain = random.uniform(.05,.1)*np.sin(2*np.pi*random.randint(17000,20000)*time) + decay[-1]

release = np.geomspace(sustain[-1], 0.001, num = release_end, endpoint = False)
plt.plot(np.concatenate((attack, decay, sustain, release)))
print(len(np.concatenate((attack, decay, sustain, release))))

In [ ]:
##make it a function -- ranges are doubles to express proportion of 
##input signal
##Thing to improve: sustain parameters could be input, particularly frequency
def random_adsr(array_length, a_range, d_range, s_range, r_range, fs):
    length = array_length

    attack_max_endpoint = length*a_range
    decay_max_endpoint = length*d_range
    sustain_max_endpoint = length*s_range
    release_max_endpoint = length*r_range

    attack_end = int(np.ceil(random.uniform(0,1)*attack_max_endpoint))
    decay_end = int(np.ceil(random.uniform(0,1)*decay_max_endpoint))
    sustain_end = int(np.ceil(random.uniform(0,1)*sustain_max_endpoint))
    release_end = length - (attack_end + decay_end + sustain_end)

    attack = np.geomspace(0.001, random.uniform(.8,1), num = attack_end, endpoint = True)
    decay = np.geomspace(attack[-1], random.uniform(.4,.8), num = decay_end, endpoint = False)

    time = np.arange(0, 1/fs*sustain_end, 1/fs)
    sustain = random.uniform(.05,.1)*np.sin(2*np.pi*random.randint(17000,20000)*time) + decay[-1]
    
    release = np.geomspace(sustain[-1], 0.001, num = release_end, endpoint = True)
    
    to_return = np.concatenate((attack, decay, sustain, release))
    
    return np.concatenate((attack, decay, sustain, release))[0:length]